In [ ]:
!pip install --upgrade openai --quiet

In [ ]:
!pip install "git+https://github.com/peterjohnlawrence/IntelligentGraph.git"

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'insert your own key here'

In [ ]:
import intelligentgraph
from intelligentgraph import IntelligentConjunctiveGraph, IntelligentDataset,IntelligentGraph,SCRIPT
from rdflib import  Literal,   URIRef
from rdflib.namespace import SDO

In [ ]:
from openai import OpenAI
import json
import os
def getCities(countryCode):
  client = OpenAI()
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "Make a json structure of three columns, one for city/town name,  one for its population, and one for its area in hectares. for example { 'results':[{'city':'Manchester','population':'530300', 'area':'4390'}]}. Format the population number without commas ','"},
      {"role": "user", "content": "Now fill in the results with the top 10 cities in the " + countryCode + " by population."}
    ]
  )
  return json.loads(completion.choices[0].message.content)
for city in getCities('UK')['results']:
  print('city='+city['city']+',population='+city['population']+',area=' +city['area']+',density='+str(float(city['population'])/float(city['area'])))

In [ ]:
def getCity(country):
  contryCode = os.path.split(country.toPython())[1]
  for city in getCities(contryCode)['results']:
    cityName =city['city'].replace(' ','_')
    cityIRI = URIRef("http://inova8.com/"+cityName)
    yield (country, SDO.City ,cityIRI)
    population = float(city['population'].replace(',',''))
    area = float(city['area'].replace(',',''))
    yield (cityIRI, URIRef("http://inova8.com/population") , Literal(population))
    yield (cityIRI, URIRef("http://inova8.com/area") , Literal(area))
    yield (cityIRI, URIRef("http://inova8.com/density") , Literal(population/area))
for triple in getCity(URIRef("http://inova8.com/France")):
  print(triple)

In [ ]:
cityg = IntelligentGraph()

USA = URIRef("http://inova8.com/USA")
cityg.add((USA, SDO.City, Literal('''
from openai import OpenAI
from rdflib.namespace import SDO
import os
import json
def getCities(countryCode):
  client = OpenAI()
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "Make a json structure of three columns, one for city/town name, one for its population, and one for its area in hectares. for example { 'results':[{'city':'Manchester','population':'530300', 'area':'4390'}]}. Format the population number without commas ','"},
      {"role": "user", "content": "Now fill in the results with the top 10 cities in the " + countryCode + " by population."}
    ]
  )
  return json.loads(completion.choices[0].message.content)
def getCity(country):
  contryCode = os.path.split(country.toPython())[1]
  for city in getCities(contryCode)['results']:
    cityName =city['city'].replace(' ','_')
    cityIRI = URIRef("http://inova8.com/"+cityName)
    yield (country, SDO.City ,cityIRI)
    population = float(city['population'].replace(',',''))
    area = float(city['area'].replace(',',''))
    yield (cityIRI, URIRef("http://inova8.com/population") , Literal(population))
    yield (cityIRI, URIRef("http://inova8.com/area") , Literal(area))
    yield (cityIRI, URIRef("http://inova8.com/density") , Literal(population/area))
_result = getCity(s)''',datatype=SCRIPT.python)))

<Graph identifier=N8543ce1ce31d4050a5b67295d9f87b3d (<class 'intelligentgraph.graph.IntelligentGraph'>)>

In [ ]:
print(cityg.serialize(format="nt"))

<http://inova8.com/USA> <https://schema.org/City> <http://inova8.com/New_York> .
<http://inova8.com/New_York> <http://inova8.com/population> "8622698.0"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://inova8.com/New_York> <http://inova8.com/area> "78217.0"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://inova8.com/New_York> <http://inova8.com/density> "110.24071493409362"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://inova8.com/USA> <https://schema.org/City> <http://inova8.com/Los_Angeles> .
<http://inova8.com/Los_Angeles> <http://inova8.com/population> "3990456.0"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://inova8.com/Los_Angeles> <http://inova8.com/area> "130060.0"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://inova8.com/Los_Angeles> <http://inova8.com/density> "30.68165462094418"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://inova8.com/USA> <https://schema.org/City> <http://inova8.com/Chicago> .
<http://inova8.com/Chicago> <http://inova8.com/p